### poc and wip for multivar regression

# Imports and setup

General imports

In [ ]:
import pandas as pd

OpenGrid-specific imports

In [ ]:
from opengrid.library import houseprint
from opengrid import config
from opengrid.library import linearregression

c = config.Config()

Plotting settings

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

# Load Data

We are going to use gas consumption data and weather data. Because we don't want to overload the weather API, we will only use 1 location (Ukkel).

First, let's define the start and end date of our experiment. Let's take 1 year worth of data, starting with last month.

In [ ]:
end = pd.Timestamp.today().replace(day=1).normalize() + pd.Timedelta(days=1)
start = (end.replace(year=end.year-1)) - pd.Timedelta(days=2)

start = start.tz_localize('Europe/Brussels')
end = end.tz_localize('Europe/Brussels')
print(start, end)

## Gas Data

In [ ]:
# Load the Houseprint, and sync all data
hp = houseprint.Houseprint()
#hp = houseprint.load_houseprint_from_file('cache_hp.hp')
hp.init_tmpo()

In [ ]:
hp.sync_tmpos()

In [ ]:
#hp.save('cache_hp.hp')

In [ ]:
def gas_data_generator4():
    # Preferred method: as accurate as 3, and faster
    # Daily approach, obtain fully correct daily data.
    # To be aggregated to monthly or weekly or ...
    
    for gas_sensor in hp.get_sensors(sensortype='gas'):
        df = gas_sensor.get_data(head=start, tail=end, resample='day', unit='kWh', diff=False, tz='Europe/Brussels')
        df = df.diff().shift(-1).dropna()
        if df.empty:
            continue
        yield df

Let's have a peek

In [ ]:
gas_data = gas_data_generator4()

In [ ]:
peek = next(gas_data)
peek.plot()

## Weather Data

Run this block to download the weather data and save it to a pickle. This is a large request, and you can only do 2 or 3 of these per day before your credit with Forecast.io runs out!

To get the data run the cell below

In [ ]:
from opengrid.library import forecastwrapper
weather = forecastwrapper.Weather(location='Ukkel, Belgium', start=start, end=end - pd.Timedelta(days=1))
weather_data = weather.days(heating_base_temperatures=[0, 6, 8 ,10, 12, 14, 16, 18]).dropna(axis=1)
weather_data.drop(['icon', 'summary'], axis=1, inplace=True)
weather_data = weather_data.applymap(float)

# Put data together

I wrote a generator that uses our previously defined generator so you can generate while you generate.

In [ ]:
def analysis_data_generator():
    gas_data = gas_data_generator4()
    for gas_df in gas_data:
        gas_df.name='gas'
        df = pd.concat([gas_df, weather_data], axis=1).dropna()
        df = df.tz_convert('Europe/Brussels')
        yield df

Let's have another peek

In [ ]:
analysis_data = analysis_data_generator()

In [ ]:
peek = next(analysis_data)
peek = peek.resample(rule='MS').sum()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot_date(peek.index, peek['gas'], '-', color='grey', lw=8, label='gas')
for column in peek.columns[1:]:
    if 'heatingDegreeDays' in column:
        ax2.plot_date(peek.index, peek[column], '-', label=column)
plt.legend()

# Run Regression Analysis



In [ ]:
%%bash
pip install statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as fm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
def find_best_rsquared(list_of_fits):
    """Return the best fit, based on rsquared"""
    res = sorted(list_of_fits, key=lambda x:x.rsquared)
    return res[-1]

def find_best_akaike(list_of_fits):
    """Return the best fit, based on Akaike information criterion"""
    res = sorted(list_of_fits, key=lambda x:x.aic)
    return res[0]
    


In [ ]:
analysis_data = analysis_data_generator()
all_HDD = weather_data.filter(like='heatingDegreeDays').columns

for data in analysis_data:  
    fits = []
    data = data.resample(rule='MS').sum()
    for HDD in all_HDD:
        fit = fm.ols(formula='gas ~ {}'.format(HDD), data=data).fit()
        fits.append(fit)
    
    # Visualise the best model
    bestfit = find_best_akaike(fits)
    HDD = list(set(bestfit.model.exog_names) - set(['Intercept'])) # this will work for multivariate models, but not the rest of this code
    data['model'] = bestfit.predict(data)
    
    plt.figure()
    plt.plot(data[HDD[0]], data['gas'], 'r*', ms=7)
    # get sorted model values
    dfmodel = data[[HDD[0], 'model']]
    dfmodel.index = dfmodel[HDD[0]]
    dfmodel.sort(inplace=True)
    plt.plot(dfmodel.index, dfmodel['model'], 'b--')
    plt.title('{} - rsquared={} - AIC={}'.format(bestfit.model.formula, bestfit.rsquared, bestfit.aic))
    plt.show()

## Try multivariable regression

In [ ]:
analysis_data = analysis_data_generator()
all_HDD = weather_data.filter(like='heatingDegreeDays').columns

for data in analysis_data:  
    fits = []
    data = data.resample(rule='MS').sum()
    for HDD in all_HDD:
        fit = fm.ols(formula='gas ~ {}'.format(HDD), data=data).fit()
        fits.append(fit)
    
    # Visualise the best model
    bestfit = find_best_akaike(fits)
    #print(bestfit.summary())
    HDD = list(set(bestfit.model.exog_names) - set(['Intercept'])) # this will work for multivariate models, but not the rest of this code
    exog_to_test = ['cloudCover', 'daytimeTemperature', 'dewPoint', 'humidity', 'pressure', 
                   'visibility', 'windBearing', 'windSpeed', 'temperatureEquivalent', 'heatingDegreeDays0']
    for exog in exog_to_test:
        fit = fm.ols(formula='gas ~ {} + {}'.format(HDD[0], exog), data=data).fit()
        fits.append(fit)
        
    bestfit = find_best_akaike(fits)
    print(bestfit.summary())
    
    data['model'] = bestfit.predict(data)
    
    plt.figure()
    plt.plot(data[HDD[0]], data['gas'], 'r*', ms=8)
    # get sorted model values
    dfmodel = data[[HDD[0], 'model']]
    dfmodel.index = dfmodel[HDD[0]]
    dfmodel.sort(inplace=True)
    plt.plot(dfmodel.index, dfmodel['model'], 'b--')
    plt.title('{} - rsquared={} - AIC={}'.format(bestfit.model.formula, bestfit.rsquared, bestfit.aic))
    plt.show()